In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from lib.rl.trajectory import serve_trajectory_batch_api

server = serve_trajectory_batch_api(None)

WARNING 11-12 15:56:47 _custom_ops.py:19] Failed to import from vllm._C with ModuleNotFoundError("No module named 'vllm._C'")
INFO 11-12 15:56:47 importing.py:10] Triton not installed; certain GPU-related functions will not be available.


/Users/brad/github/bradhilton/atreides/.venv/lib/python3.12/site-packages/vllm/connections.py:8: RuntimeWarning: Failed to read commit hash:
No module named 'vllm._version'
  from vllm.version import __version__ as VLLM_VERSION


INFO:     Started server process [11847]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://127.0.0.1:8000 (Press CTRL+C to quit)


INFO:     127.0.0.1:64969 - "GET / HTTP/1.1" 200 OK
INFO:     127.0.0.1:64970 - "GET / HTTP/1.1" 200 OK
INFO:     127.0.0.1:64971 - "GET / HTTP/1.1" 200 OK
INFO:     127.0.0.1:64972 - "GET / HTTP/1.1" 200 OK
INFO:     127.0.0.1:64973 - "GET / HTTP/1.1" 200 OK
INFO:     127.0.0.1:64974 - "GET / HTTP/1.1" 200 OK
INFO:     127.0.0.1:64975 - "GET / HTTP/1.1" 200 OK
INFO:     127.0.0.1:64976 - "GET / HTTP/1.1" 200 OK
INFO:     127.0.0.1:64977 - "GET / HTTP/1.1" 200 OK
INFO:     127.0.0.1:64978 - "GET / HTTP/1.1" 200 OK
INFO:     127.0.0.1:64979 - "GET / HTTP/1.1" 200 OK
INFO:     127.0.0.1:64980 - "GET / HTTP/1.1" 200 OK
INFO:     127.0.0.1:64981 - "GET / HTTP/1.1" 200 OK
INFO:     127.0.0.1:64982 - "GET / HTTP/1.1" 200 OK
INFO:     127.0.0.1:64983 - "GET / HTTP/1.1" 200 OK
INFO:     127.0.0.1:64984 - "GET / HTTP/1.1" 200 OK
INFO:     127.0.0.1:64985 - "GET / HTTP/1.1" 200 OK


In [4]:
import os

os.getpid()

11847

In [3]:
import requests

requests.get("http://localhost:8000/")

KeyboardInterrupt: 

In [3]:
from lib.rl.trajectory import TrajectoryBatchRequest
import requests

response = requests.post(
    "http://localhost:8000/trajectory-batch",
    json=TrajectoryBatchRequest(
        dir="./tensors", rows=1_000, seqlen=8192, start=0, stop=10
    ).model_dump(mode="json"),
    timeout=5,
)
response

KeyboardInterrupt: 

In [6]:
from IPython.display import HTML
from lib.rl import Completion, Episode
from lib.tokenizer import Tokenizer

tokenizer = Tokenizer("NousResearch/Hermes-2-Theta-Llama-3-8B")
completion = Completion.model_validate_json(open("best_trajectory.json").read())
episode = Episode([], lambda _: None)
episode.completion = completion
terminus = episode.best_leaf(tokenizer=tokenizer)

INFO 11-08 20:01:28 llm_engine.py:237] Initializing an LLM engine (vdev) with config: model='NousResearch/Hermes-2-Theta-Llama-3-8B', speculative_config=None, tokenizer='NousResearch/Hermes-2-Theta-Llama-3-8B', skip_tokenizer_init=False, tokenizer_mode=auto, revision=None, override_neuron_config=None, rope_scaling=None, rope_theta=None, tokenizer_revision=None, trust_remote_code=False, dtype=torch.bfloat16, max_seq_len=8192, download_dir=None, load_format=LoadFormat.AUTO, tensor_parallel_size=1, pipeline_parallel_size=1, disable_custom_all_reduce=False, quantization=None, enforce_eager=False, kv_cache_dtype=auto, quantization_param_path=None, device_config=cuda, decoding_config=DecodingConfig(guided_decoding_backend='outlines'), observability_config=ObservabilityConfig(otlp_traces_endpoint=None, collect_model_forward_time=False, collect_model_execute_time=False), seed=0, served_model_name=NousResearch/Hermes-2-Theta-Llama-3-8B, use_v2_block_manager=True, num_scheduler_steps=1, chunked_

In [29]:
HTML(f'<div style="white-space: pre-wrap">{terminus.html()}</div>')

In [24]:
for leaf in completion.leaves():
    if leaf.value() == 1.0:
        display(HTML(f'<div style="white-space: pre-wrap">{leaf.html()}</div>'))
        break

In [27]:
leaf.advantage()

0.2944809993108114